In [6]:
import os
import pandas as pd
import numpy as np
import pyBigWig
import scanpy as sc

os.chdir("/media/kyle_storage/kyle_ferchen/grimes_lab_main/analysis/"\
    "2023_06_12_tea_seq_atac_processing/")

In [7]:
# Read in the peak locations for processing
path_peak_nominations = "output/"\
    "correlate_tea_atac_to_cite_rna_across_r7_clusters/"\
    "peak_to_gene_correlation_within_tads/sig_conns_df.csv"

peak_anno = pd.read_csv(path_peak_nominations)
peak_anno["chr"] = [i.split(":")[0] for i in peak_anno.iloc[:,2].values]
peak_anno["start"] = [int(i.split(":")[-1].split("-")[0]) for i in peak_anno.iloc[:,2].values]
peak_anno["end"] = [int(i.split("-")[-1]) for i in peak_anno.iloc[:,2].values]
peak_anno

,peak,gene,peak.1,r,p,chr,start,end
0,23,2900089D17Rik,chr5:144026531-144027531,0.689254,8.454730e-08,chr5,144026531,144027531
1,32,2900089D17Rik,chr5:144040343-144041343,0.466576,9.470651e-04,chr5,144040343,144041343
2,59,2900089D17Rik,chr5:144099829-144100829,0.503086,3.131146e-04,chr5,144099829,144100829
3,60,2900089D17Rik,chr5:144099935-144100935,0.503974,3.043190e-04,chr5,144099935,144100935
4,72,2900089D17Rik,chr5:144125641-144126641,0.551664,5.836859e-05,chr5,144125641,144126641
...,...,...,...,...,...,...,...,...
236943,335,Lypd6,chr2:50174219-50175219,0.538422,9.466431e-05,chr2,50174219,50175219
236944,337,Lypd6,chr2:50177952-50178952,0.784699,6.704674e-11,chr2,50177952,50178952
236945,339,Lypd6,chr2:50188001-50189001,0.485716,5.383332e-04,chr2,50188001,50189001
236946,352,Lypd6,chr2:50265457-50266457,0.494948,4.050589e-04,chr2,50265457,50266457


In [8]:
# Get bigwig files and make names for those bigwig files
path_to_bigwigs = "output/chrombpnet/browser_visualization/"\
    "count_scores_bw_extended_peak_set/fold_0/"

list_bw_files = os.listdir(path_to_bigwigs)

list_sample_names = [i.replace(".bw","") for i in list_bw_files]

list_sample_names[:5]

['BMCP', 'CD127_MP', 'CLP1_Rrm2', 'eHSC', 'eHSC_Pcna']

In [9]:
# Read in bigwig files
bw_dict = {}
for tmp_name, tmp_file in zip(list_sample_names, list_bw_files):
    bw_dict[tmp_name] = pyBigWig.open(os.path.join(path_to_bigwigs, tmp_file))

In [10]:
# Define a window size (constant size for seqlets)
window_size = 20
# Threshold for average contribution score over the window
cs_thresh = 0.005

region_anno = []
for i in range(peak_anno.head(1000).shape[0]):
    print(f"{i+1}/{peak_anno.shape[0]} peaks processed...", 
        end="\r", flush=True)
    # Store tmp values for current peak
    tmp_chr, tmp_start, tmp_end, tmp_gene = peak_anno[["chr", "start", "end", "gene"]].iloc[i,:4].values
    tmp_peak_size = tmp_end - tmp_start

    # Extract bigwig values
    test = pd.DataFrame(\
        [bw_dict[i].values(tmp_chr, tmp_start, tmp_end) for i in list_sample_names],
        index=list_sample_names)

    # Get window means
    test = test.rolling(\
        window=window_size, 
        min_periods=window_size, 
        axis=1).mean().iloc[:,window_size-1:]
    test.columns = list(range(tmp_peak_size-window_size+1))

    # Identify non-overlapping highest scoring regions
    list_i = test.abs().max(axis=0).sort_values(ascending=False)
    list_i = list_i[list_i > cs_thresh].index.values
    saved_i = []
    while(len(list_i) > 0):
        i_sel = list_i[0]
        saved_i.append(i_sel)
        tmp_check = list_i - i_sel
        list_i = list_i[(tmp_check < (-1*window_size)) | (tmp_check > window_size)]

    # Save footprints for the current peak
    region_anno.append(pd.DataFrame({\
            "chr": tmp_chr,
            "start": tmp_start + np.array(saved_i),
            "end": tmp_start + np.array(saved_i) + window_size - 1,
            "gene": tmp_gene,
            "peak": f"{tmp_chr}:{tmp_start}-{tmp_end}",
            "offset": saved_i,
            "max_cs": test.loc[:,saved_i].max(axis=0).values,
            "max_cs_group": test.loc[:,saved_i].idxmax(axis=0).values,
            "min_cs": test.loc[:,saved_i].min(axis=0).values,
            "min_cs_group": test.loc[:,saved_i].idxmin(axis=0).values},
        index=saved_i))

region_anno = pd.concat(region_anno)


In [13]:
# Close the bigwig files
bw_dict = {}
for tmp_name in bw_dict:
    bw_dict[tmp_name].close()

In [11]:
region_anno

,chr,start,end,gene,peak,offset,max_cs,max_cs_group,min_cs,min_cs_group
485,chr5,144027016,144027035,2900089D17Rik,chr5:144026531-144027531,485,0.033448,MultiLin_1,-0.001832,HSCP_ERP1
538,chr5,144027069,144027088,2900089D17Rik,chr5:144026531-144027531,538,0.031800,LT_HSC_Mllt3,0.002988,HSCP_ERP1
593,chr5,144027124,144027143,2900089D17Rik,chr5:144026531-144027531,593,0.025088,MultiLin_1,0.001312,ERP2
458,chr5,144026989,144027008,2900089D17Rik,chr5:144026531-144027531,458,0.022766,MultiLin_1,0.002269,ST_HSC
675,chr5,144027206,144027225,2900089D17Rik,chr5:144026531-144027531,675,0.016499,MEP,0.002885,proNeu_1
...,...,...,...,...,...,...,...,...,...,...
384,chr2,131072400,131072419,Pcna,chr2:131072016-131073016,384,-0.001171,CLP1_Rrm2,-0.005587,HSCP_HPC_Cenpf
882,chr2,131072898,131072917,Pcna,chr2:131072016-131073016,882,-0.000651,BMCP,-0.005420,ERP1
107,chr2,131072123,131072142,Pcna,chr2:131072016-131073016,107,-0.001267,MultiLin_1,-0.005328,MPP5_Egr1
210,chr2,131072226,131072245,Pcna,chr2:131072016-131073016,210,0.001652,MKP,-0.005260,eHSC_Pcna


In [ ]:
# Save all regions
path_output = "output/chrombpnet/seqlets/expanded_peak_set_fold_0/"
region_anno.to_csv(os.path.join(path_output, "seqlet_regions_absolute_cs_above_0_005.csv"),
    header=True, index=False)